# Sama app
**author:** _Razan Aljuraysi_

**group:** 1

**notes:**
- First create an account
- Login with the account you have created
- Then you can use the app as you like

.

- If it was an admin account then this would be the passcode 121212

.

- Running the code again will reset everything. Therefore, logging out and logging in again without terminating the kernel is recommended.

.
- Some functions might not work perfectly due to time constraints and limited testing.

In [ ]:
# imports
import csv
import sys

In [ ]:

# empty database for users
users_db = []

#  user and admin classes
class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.is_admin = False

class Admin(User):
    def __init__(self, username, password): # Session id
        super().__init__(username, password)
        self.is_admin = True


### LOGIN OR SIGNUP

In [ ]:
# Sign up
def signup():
    print("Sign Up")
    while True:
        username = input("Enter your username: ")
        password = input("Enter your password: ")
        confirm_password = input("Confirm your password: ")

        # Check if the username is already taken
        if any(user.username == username for user in users_db):
            print("Username is already taken. Please choose another username.")
            continue

        if password == confirm_password:
            # is it an admin account ?
            is_admin = input("Do you want to create an admin account? (yes/no): ").lower()

            if is_admin == "yes":
                admin_password = input("Enter the admin password: ")

                # Check if the admin password is correct
                if admin_password == "121212": # basiacally this is a password that is passed to the admin from the developer, so in case we have many admins they can create an account for themselves and use this password 
                    user = Admin(username, password)  # Create an Admin object
                else:
                    print("Admin password is incorrect. Admin account creation failed.")
                    continue
            else:
                user = User(username, password)  # Create a User object

            users_db.append(user)
            print("Sign up successful!")

            # Write user data to a CSV file 
            with open("users.csv", "a", newline="") as file:
                writer = csv.writer(file)
                if not file.tell():  
                    writer.writerow(["username", "password", "is_admin"])
                writer.writerow([user.username, user.password, user.is_admin])

            break
        else:
            print("Passwords do not match. Try again.")




# log in
def login():
    print("Log In")
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    for user in users_db:
        if user.username == username and user.password == password:
            if user.is_admin:
                admin_menu(user)  # If user is an admin, show admin menu
            else:
                user_menu(user)    # If user is a regular user, show user menu
            return

    print("Invalid username or password. Please try again.")



### USER

In [ ]:
import csv

def get_unique_categories():
    categories = set()  # Use a set to store unique category names
    with open("product.csv", "r", newline="") as file:
        reader = csv.DictReader(file)
        for row in reader:
            categories.add(row["category_name"])
    return list(categories)

# A dictionary for the user's shopping cart
user_carts = {}

def view_products(category):
    while True:  # Add a loop to ask for more items
        with open("product.csv", "r", newline="") as file:
            reader = csv.DictReader(file)
            products = [row for row in reader if row["category_name"] == category]

        print(f"{category.capitalize()} Products:")
        for i, product in enumerate(products, start=1):
            print(f"{i}. {product['Product']} {product['Price']} {product['Currency']}")

        while True:  # Add a loop to ask for more items from the same category
            choice = input(f"Enter the number of the product you want to add to your cart (1-{len(products)}): ")

            if choice.isdigit():
                choice = int(choice)
                if 1 <= choice <= len(products):
                    selected_product = products[choice - 1]["Product"]
                    quantity = input(f"Enter the quantity for '{selected_product}': ")

                    if selected_product in user_carts:
                        user_carts[selected_product] += int(quantity)
                    else:
                        user_carts[selected_product] = int(quantity)

                    print(f"{selected_product} (Quantity: {quantity}) added to your cart.")
                else:
                    print("Invalid choice. Please enter a valid number.")
            else:
                print("Invalid choice. Please enter a valid number.")

            while True:
                add_more = input(f"Do you want to add more items from {category.capitalize()}? (yes/no): ").lower()
                if add_more not in ["yes", "no"]:
                    print("Invalid choice. Please choose 'yes' or 'no'.")
                else:
                    break  
            if add_more == "no":
                return  # Exit the function if the user doesn't want to add more items from the same category

def view_categories():
    while True:  # Add a loop to return to category
        print("Available Categories:")
        categories = get_unique_categories()  
        for i, category in enumerate(categories, start=1):
            print(f"{i}. {category}")

        choice = input(f"Enter the number of the category you want to view (1-{len(categories)}) or '0' to go back: ")

        if choice.isdigit():
            choice = int(choice)
            if 1 <= choice <= len(categories):
                selected_category = categories[choice - 1]
                view_products(selected_category)  # Call view_products with the selected category
            elif choice == 0:
                return  
            else:
                print("Invalid choice. Please enter a valid number.")
        else:
            print("Invalid choice. Please enter a valid number.")


In [ ]:

def manage_cart():
    while True:
        print("\nCart Menu:")
        print("1. View Cart")
        print("2. Add More Items")
        print("3. Remove Items")
        print("4. Checkout")
        print("5. Back to Main Menu")
        choice = input("Enter your choice (1/2/3/4/5): ")

        if choice == '1':
            view_cart()
        elif choice == '2':
            view_categories()
        elif choice == '3':
            remove_items()
        elif choice == '4':
            checkout()  
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5.")


def view_cart():
    print("Your Cart:")
    total_cost = 0  
    
    for product, quantity in user_carts.items():
        with open("product.csv", "r", newline="") as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["Product"] == product:
                    price = float(row["Price"])
                    item_total = price * quantity
                    total_cost += item_total
                    print(f"{product} (Quantity: {quantity}) : {item_total} sr")

    print("----------------------------------------")
    print(f"total : {total_cost} sr")


        
def remove_items():
    print("Remove Items from Cart:")
    view_cart()
    item_to_remove = input("Enter the name of the item you want to remove: ").strip().lower()
    
    if item_to_remove in user_carts:
        del user_carts[item_to_remove]
        print(f"{item_to_remove} removed from your cart.")
    else:
        print(f"{item_to_remove} not found in your cart.")

In [ ]:
user_payments = {}
def checkout():
    view_cart()  # Display the cart before checkout
    total_cost = calculate_total_cost()  # Calculate the total cost of items in the cart
    
    print(f"Total cost: {total_cost} sr")
    
    while True:
        choice = input("Choose a payment type (1: Credit Card / 2: Cash / 3: Go Back to Cart): ")
        
        if choice == '1':
            pay_with_credit_card(total_cost)
            break
        elif choice == '2':
            pay_with_cash(total_cost)
            break
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please choose 1, 2, or 3.")

            
def calculate_total_cost():
    total_cost = 0
    for product, quantity in user_carts.items():
        with open("product.csv", "r", newline="") as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row["Product"] == product:
                    price = float(row["Price"])
                    item_total = price * quantity
                    total_cost += item_total
    return total_cost


def pay_with_credit_card(total_cost):
    print('assume that you have already been transferred to an external portal and added your credit card details')
    print(f"Payment of {total_cost} sr made with Credit Card.")
    user_payments["Credit Card"] = total_cost
    print('Thank you for using Sama App, see you next time!')
    sys.exit()  

def pay_with_cash(total_cost):
    print(f"Payment of {total_cost} sr will be made after delivery ")
    user_payments["Cash"] = total_cost
    print('Thank you for using Sama App, see you next time!')
    sys.exit()  





### ADMIN

In [ ]:
def get_next_product_id():
    existing_ids = set()
    with open("product.csv", "r", newline="") as file:
        reader = csv.DictReader(file)
        for row in reader:
            existing_ids.add(int(row["id"]))


    next_id = 1
    while next_id in existing_ids:
        next_id += 1

    return next_id

In [ ]:
def modify_product():
    while True:
        print("\nModify Product Menu:")
        print("1. Add Product")
        print("2. Remove Product")
        print("3. Back to Admin Menu")
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            add_product()
        elif choice == '2':
            remove_product()
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3")

def add_product():
    print("Add Product:")
    product_name = input("Enter the product name: ")
    category_name = input("Enter the category name: ")
    price = input("Enter the price: ")
    currency = input("Enter the currency: ")

    with open("product.csv", "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([category_name, product_name, price, currency, get_next_product_id()])

    print(f"{product_name} added to the product list.")

def remove_product():
    print("Remove Product:")
    product_name = input("Enter the name of the product you want to remove: ")

    products = []
    with open("product.csv", "r", newline="") as file:
        reader = csv.reader(file)
        for row in reader:
            products.append(row)

    removed = False
    updated_products = []
    for product in products:
        if product[0].strip().lower() == product_name.strip().lower():
            print(f"{product[0]} removed from the product list.")
            removed = True
        else:
            updated_products.append(product)

    if removed:
        with open("product.csv", "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerows(updated_products)
    else:
        print(f"{product_name} not found in the product list.")
        print('this function will be modified later on ') # COME BACK

def modify_category():
    while True:
        print("\nModify Category Menu:")
        print("1. Add Category")
        print("2. Remove Category")
        print("3. Back to Admin Menu")
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            add_category()
        elif choice == '2':
            remove_category()
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3")

def add_category():
    print('for the meantime you can only add a new category through the product section') # COME BACK
    # print("Add Category:")
#     category_name = input("Enter the category name: ")

#     # Check if the category already exists in the product.csv file
#     categories = set()
#     with open("product.csv", "r", newline="") as file:
#         reader = csv.reader(file)
#         next(reader)  # Skip the header row
#         for row in reader:
#             categories.add(row[0])

#     if category_name in categories:
#         print(f"{category_name} already exists.")
#     else:
#         with open("product.csv", "a", newline="") as file:
#             writer = csv.writer(file)
#             writer.writerow([category_name, "", "", "", ""])  # Add an empty category row

#         print(f"{category_name} added to the category list.")


def remove_category():
    print('for the meantime you can only remove categories manualy') # COME BACK

#     print("Remove Category:")
#     category_name = input("Enter the name of the category you want to remove: ")

#     categories = []
#     products = []
#     with open("product.csv", "r", newline="") as file:
#         reader = csv.reader(file)
#         header = next(reader)  # Read the header row
#         for row in reader:
#             if row[0].strip().lower() == category_name.strip().lower():
#                 print(f"{row[0]} removed from the category list.")
#             else:
#                 products.append(row)
#                 categories.add(row[0])

#     with open("product.csv", "w", newline="") as file:
#         writer = csv.writer(file)
#         writer.writerow(header)
#         writer.writerows(products)

#     if not categories:
#         print("No products left in the removed category.")

def view_user_accounts():
    print("User Accounts:")
    try:
        with open("users.csv", "r", newline="") as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header row
            for row in reader:
                username = row[0]
                is_admin = row[2]
                user_type = "Admin" if is_admin == 'True' else "User"
                print(f"Username: {username}, Type: {user_type}")
    except FileNotFoundError:
        print("No user accounts found.")

### main functions

In [ ]:
# Admin menu
def admin_menu(user):
    print(f"Welcome, {user.username}!")
    while True:
        print("\nAdmin Menu:")
        print("1. Modify Product")
        print("2. Modify Category")
        print("3. View Orders")
        print("4. View User Accounts")
        print("5. Logout")
        choice = input("Enter your choice (1/2/3/4/5): ")

        if choice == '1':
            modify_product()
        elif choice == '2':
            modify_category()
        elif choice == '3':
            print('This feature is not available yet')
        elif choice == '4':
            view_user_accounts()
        elif choice == '5':
            print("Logging out...")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5")






# User menu
def user_menu(user):
    print(f"Welcome, {user.username}!")
    while True:
        print("\nUser Menu:")
        print("1. View Categories")
        print("2. View Shopping Cart")
        print("3. Logout")
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            view_categories()
        elif choice == '2':
            manage_cart()
        elif choice == '3':
            print("Logging out...")
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")




In [1]:
## Main menu
def main():
    print('''                                    ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒                                      
                                        ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒                                    
                                      ▒▒░░▓▓▓▓░░░░▒▒  ▒▒  ▓▓▓▓▓▓  ▓▓▓▓▓▓░░░░                                    
                        ████████▓▓▓▓██░░  ▓▓░░    ▒▒  ▒▒  ▒▒  ██  ▓▓░░░░░░  ▓▓▓▓████████▓▓                      
                      ▒▒░░░░░░░░░░░░░░  ░░▓▓▒▒░░  ▓▓▒▒▓▓░░▒▒░░██░░▓▓▒▒▓▓░░░░░░░░░░░░░░░░░░▒▒                    
                      ▓▓░░░░  ░░░░░░░░░░░░░░▒▒▒▒░░▒▒  ▓▓░░▓▓▒▒██░░▓▓░░░░░░░░    ░░░░░░░░░░▓▓                    
                      ▓▓░░░░░░░░  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓                    
                      ▒▒░░▒▒▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓▓▓▒▒░░▒▒▒▒▒▒▓▓░░▒▒▒▒▒▒▓▓░░░░▓▓▒▒▒▒░░                  
                    ░░▒▒░░▒▒▒▒▒▒▓▓░░▒▒▓▓▒▒▒▒▓▓░░▒▒▒▒▒▒▒▒▓▓▓▓▒▒░░▒▒▒▒▒▒▒▒░░▒▒▒▒▒▒▓▓▒▒░░▓▓▒▒▒▒░░                  
                    ▓▓░░░░▓▓▒▒▒▒░░░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒██░░▓▓▒▒▓▓░░▒▒▒▒▒▒▒▒░░░░▓▓▒▒▒▒▓▓░░▒▒▒▒▒▒▓▓                  
                  ▓▓▒▒░░▓▓▒▒▒▒▓▓░░▒▒▒▒▒▒▓▓░░░░▒▒▒▒▒▒▓▓░░▓▓▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒░░░░▓▓▒▒▒▒▒▒                
               ██▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓                
                ██▒▒░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒░░░░░░▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░▒▒▒▒▒▒▓▓░░░░▓▓▒▒▓▓                
                ██▒▒░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒░░░░░░▓▓▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░▒▒▒▒▒▒▓▓░░░░▓▓▒▒▓▓                
                ░░▓▓▓▓▓▓▓▓▓▓▒▒▓▓▓▓▒▒▓▓▓▓▓▓▓▓▓▓▒▒▓▓▒▒▓▓▓▓░░▓▓▒▒░░▓▓▓▓▒▒▓▓▓▓▒▒▓▓▒▒▓▓▓▓░░▓▓▓▓▓▓▓▓▒▒                
                      ▓▓░░▓▓░░▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒▓▓░░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ██░░▓▓░░░░░░░░░░▓▓▒▒▓▓░░  ▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░░░▓▓░░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓                    
                      ▓▓░░▓▓░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░▒▒                    
                      ▓▓░░▓▓░░░░░░░░░░░░▒▒██░░░░▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓                    
                      ▓▓░░▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓░░░░░░░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▒▒▒▒░░▓▓                    
                ░░░░░░▓▓░░░░░░░░  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░                
                ▒▒▒▒░░░░░░▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▓▓              
              ░░▒▒░░▒▒░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒██              
              ░░░░░░░░▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒██              
                ██████▓▓▓▓████▓▓▓▓▓▓▓▓▓▓████▓▓▓▓▓▓▓▓▓▓██████████████████▓▓▓▓██████████████▓▓████                

''')
    print("Welcome to!")               
    print('''        
                     __    __    _       __        _       __    ___   _     ____ _____ 
                    ( (`  / /\  | |\/|  / /\      | |\/|  / /\  | |_) | |_/ | |_   | |  
                    _)_) /_/--\ |_|  | /_/--\     |_|  | /_/--\ |_| \ |_| \ |_|__  |_|  
''')
    while True:
        print("\nMain Menu:")
        print("1. Sign Up")
        print("2. Log In")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            signup()
        elif choice == '2':
            login()
        elif choice == '3':
            print("Thank you for Shopping with us!")
            break
        else:
            print("Invalid choice. Please choose 1, 2 or 3")

if __name__ == "__main__":
    main()

                                    ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒                                      
                                        ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒                                    
                                      ▒▒░░▓▓▓▓░░░░▒▒  ▒▒  ▓▓▓▓▓▓  ▓▓▓▓▓▓░░░░                                    
                        ████████▓▓▓▓██░░  ▓▓░░    ▒▒  ▒▒  ▒▒  ██  ▓▓░░░░░░  ▓▓▓▓████████▓▓                      
                      ▒▒░░░░░░░░░░░░░░  ░░▓▓▒▒░░  ▓▓▒▒▓▓░░▒▒░░██░░▓▓▒▒▓▓░░░░░░░░░░░░░░░░░░▒▒                    
                      ▓▓░░░░  ░░░░░░░░░░░░░░▒▒▒▒░░▒▒  ▓▓░░▓▓▒▒██░░▓▓░░░░░░░░    ░░░░░░░░░░▓▓                    
                      ▓▓░░░░░░░░  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓                    
                      ▒▒░░▒▒▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓░░░░▓▓▒▒▒▒▓▓▓▓▒▒░░▒▒▒▒▒▒▓▓░░▒▒▒▒▒▒▓▓░░░░▓▓▒▒▒▒░░                  
                    ░░▒▒░░▒▒▒▒▒▒▓▓░░▒▒▓▓▒▒▒▒▓▓░░▒▒▒▒▒▒▒▒▓▓▓▓▒▒░░▒▒▒▒▒▒▒▒░░▒▒▒▒▒▒▓▓▒▒░░▓▓▒▒▒▒░░      

KeyboardInterrupt: Interrupted by user

## 